In [45]:
!pip install dill

  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)


In [81]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [82]:
import pandas as pd
import dill
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import itertools

import matplotlib.pyplot as plt

%matplotlib inline

Датасет с платформы kaggle https://www.kaggle.com/fedesoriano/heart-failure-prediction

In [83]:
df = pd.read_csv("heart.csv")
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


In [94]:
df['MaxHR'].value_counts()

150    43
140    41
120    36
130    33
160    25
       ..
177     1
187     1
192     1
194     1
60      1
Name: MaxHR, Length: 119, dtype: int64

In [84]:
df['HeartDisease'].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [85]:
df['FastingBS'].value_counts()

0    704
1    214
Name: FastingBS, dtype: int64

In [71]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['HeartDisease'], random_state=0)

In [72]:
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [73]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

Зададим списки признаков

In [74]:
#categorical_columns = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
continuous_columns = ['Age', 'RestingBP', 'MaxHR', 'Oldpeak']

Теперь нам нужно под каждый признак создать трансформер и объединить их в список (сделаем это в цикле, чтобы не мучиться)

In [86]:
#combine
Age = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
             ])
RestingBP = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
             ])
MaxHR = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
             ])

Oldpeak = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
             ])

feats = FeatureUnion([('Age', Age),
                      ('RestingBP', RestingBP),
                      ('MaxHR', MaxHR),
                     ('Oldpeak', Oldpeak)])

Объединим все это в единый пайплайн

In [87]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

feature_processing = Pipeline([('feats', feats)])


In [88]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(n_estimators = 200,random_state = 42)),
])

Обучим модель

In [89]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Oldpeak')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('RestingBP',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='Oldpeak')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('MaxHR',
                                                 Pipeline(steps=[('selector',
                                 

In [90]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('Age',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='Oldpeak')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('RestingBP',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='Oldpeak')),
                                                  ('standard',
                                                   StandardScaler())])),
                                 ('MaxHR',
                                  Pipeline(steps=[('selector',
                                                   NumberSelector(key='Oldpeak')),
                                                  ('standard',
                                                   StandardScaler())])),
         

In [91]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)